In [7]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
#loading train
df_train=pd.read_csv('./cleaned_train_dummy.csv')
#loading test
df_test=pd.read_csv('cleaned_test_dummy.csv')

In [9]:
##Setting index as ID columns
df_test.set_index(['Id'], inplace = True)
df_train.set_index(['Id'], inplace = True)

In [10]:
##Setting y and x to begin running models
Y_vals = pd.DataFrame(df_train['SalePrice']).copy()
Y_vals = np.ravel(Y_vals)
del df_train['SalePrice']
del df_test['SalePrice']
X_train=df_train.copy()
X_test=df_test.copy()


In [11]:

from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr, ExtraTreesRegressor as etr
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [44]:
##Grid search for optimal random tree parameters
grid_para_lasso = [{
    "n_alphas": range(100,1000, 100),
    "random_state": [0],
    "eps" : [0.0001],
    "normalize" : [True]
}]
grid_search_lasso = GridSearchCV(lasso_model, grid_para_lasso, cv=5, n_jobs=-1, verbose=1)
%time grid_search_lasso.fit(X_train, Y_vals)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.0min finished


CPU times: user 8.19 s, sys: 295 ms, total: 8.48 s
Wall time: 1min 2s


GridSearchCV(cv=5, error_score='raise',
       estimator=LassoCV(alphas=None, copy_X=True, cv=None, eps=0.0001, fit_intercept=True,
    max_iter=10000, n_alphas=100, n_jobs=1, normalize=True, positive=False,
    precompute='auto', random_state=0, selection='cyclic', tol=0.0001,
    verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_alphas': range(100, 1000, 100), 'random_state': [0], 'eps': [0.0001], 'normalize': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [76]:
print(grid_search_lasso.best_params_)
print(grid_search_lasso.best_score_)


{'eps': 0.0001, 'n_alphas': 100, 'normalize': True, 'random_state': 0}
0.870755275974


In [77]:
lasso_model = linear_model.LassoCV(random_state=0,n_alphas=100, normalize=True, eps = 0.0001, max_iter=10000)
mod = lasso_model.fit(X_train, Y_vals)
mod.score(X_train, Y_vals)
##score=0.93082405005697799

0.93082405005697799

In [78]:
lasso_result=round(pd.DataFrame(np.exp(lasso_reg.predict(X_test))),3)
lasso_result=lasso_result.set_index(X_test.index)
lasso_result.columns = ['SalePrice']
lasso_result.to_csv('lasso_model.csv')

In [67]:
##Grid search for optimal random tree parameters
grid_para_elastic = [{
    "n_alphas": range(100,1000, 100),
    "l1_ratio": np.linspace(0.1,1,10),
    "eps" : [0.0001],
    "normalize" : [True]
}]
grid_search_elastic = GridSearchCV(elastic_model, grid_para_elastic, cv=5, n_jobs=-1, verbose=1)
%time grid_search_elastic.fit(X_train, Y_vals)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  5.3min finished


CPU times: user 10.5 s, sys: 247 ms, total: 10.7 s
Wall time: 5min 19s


GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.0001,
       fit_intercept=True, l1_ratio=0.5, max_iter=10000, n_alphas=100,
       n_jobs=1, normalize=True, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_alphas': range(100, 1000, 100), 'l1_ratio': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 'eps': [0.0001], 'normalize': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [68]:
print(grid_search_elastic.best_params_)
print(grid_search_elastic.best_score_)

{'eps': 0.0001, 'l1_ratio': 0.30000000000000004, 'n_alphas': 500, 'normalize': True}
0.878730752753


In [79]:
elastic_result=round(pd.DataFrame(np.exp(elastic_reg.predict(X_test))),3)
elastic_result=elastic_result.set_index(X_test.index)
elastic_result.columns = ['SalePrice']
elastic_result.to_csv('elastic_model.csv')

In [73]:
from sklearn.metrics import mean_squared_error

elastic_model = linear_model.ElasticNetCV(n_alphas=500, l1_ratio=0.3, normalize=True,eps = 0.0001,max_iter=10000).fit(X_train, np.ravel(Y_vals))
mse = mean_squared_error(Y_vals, elastic_model.predict(X_train))
elastic_model.score(X_train, np.ravel(Y_vals))

#print(np.sqrt(mse))

0.92389378888691109

In [64]:
##Grid search for optimal random tree parameters
from sklearn.model_selection import GridSearchCV
grid_para_rfr = [{
    "n_estimators": [1000],
    "max_depth": range(4,10),
    "min_samples_leaf": range(1, 5),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [0]
}]
grid_search_rfr = GridSearchCV(rfr_model, grid_para_rfr, cv=5, n_jobs=-1, verbose=1)
%time grid_search_rfr.fit(X_train, np.ravel(Y_vals))

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.9min finished


CPU times: user 8.1 s, sys: 244 ms, total: 8.34 s
Wall time: 4min 59s


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=True, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [1000], 'max_depth': range(4, 10), 'min_samples_leaf': range(1, 5), 'min_samples_split': array([ 2,  9, 16, 23, 30]), 'random_state': [0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [52]:
grid_search_rfr.best_params_
#{'max_depth': 9,
# 'min_samples_leaf': 1,
# 'min_samples_split': 2,
# 'n_estimators': 1000,
# 'random_state': 0}

NameError: name 'grid_search_rfr' is not defined

In [67]:
grid_search_rfr.best_score_
#0.85469381411571965

0.85469381411571965

In [69]:
print("The training error is: %.5f" % (1 - grid_search_rfr.score(X_train, Y_vals)))
#print("The test     error is: %.5f" % (1 - grid_search_rfr.score(X_test, y_test)))

The training error is: 0.05750


In [53]:
rfr_model = rfr(random_state=0, min_samples_leaf = 2,
        n_estimators=1000, max_depth=9,  max_features='sqrt', oob_score=True).fit(X_train, np.ravel(Y_vals))


In [54]:
print(rfr_model.score(X_train, np.ravel(Y_vals)))
print(rfr_model.oob_score_)


0.928247748584
0.854329848774


In [79]:
rfr_results=round(pd.DataFrame(np.exp(rfr_model.predict(X_test))),3)
rfr_results.columns=['SalePrice']
rfr_results.set_index(df_test.index, inplace=True)
rfr_results.to_csv('rfr_model.csv')

In [79]:
##Grid search for extra random tree parameters
from sklearn.model_selection import GridSearchCV
grid_para_etr = [{
    "n_estimators": [1000],
    "max_depth": range(4,10),
    "min_samples_leaf": range(1, 5),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "random_state": [0]
}]
grid_search_etr = GridSearchCV(etr_model, grid_para_etr, cv=5, n_jobs=-1, verbose=1)
%time grid_search_etr.fit(X_train, np.ravel(Y_vals))

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.2min finished


CPU times: user 7.07 s, sys: 232 ms, total: 7.3 s
Wall time: 4min 15s


GridSearchCV(cv=5, error_score='raise',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=9,
          max_features='sqrt', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
          oob_score=True, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [1000], 'max_depth': range(4, 10), 'min_samples_leaf': range(1, 5), 'min_samples_split': array([ 2,  9, 16, 23, 30]), 'random_state': [0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [ ]:
print(grid_search_etr.best_params_)
print(grid_search_etr.best_score_)
#{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 0}
#0.81466266301

In [88]:
etr_model = etr(random_state=0,
        n_estimators=1000, max_depth=9, min_samples_leaf= 1, max_features='sqrt', bootstrap=True,oob_score=True).fit(X_train,Y_vals)


In [89]:
etr_results=round(pd.DataFrame(np.exp(etr_model.predict(X_test))),3)
etr_results.columns=['SalePrice']
etr_results.set_index(df_test.index, inplace=True)
etr_results.to_csv('etr_model.csv')

In [90]:
print(etr_model.score(X_train, Y_vals))
print(etr_model.oob_score_)

0.901928957492
0.81472331235


In [23]:
n_folds = 5
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
scorer = make_scorer(mean_squared_error,greater_is_better = False)
def rmse_CV_train(model):
    kf = KFold(n_folds,shuffle=True,random_state=42).get_n_splits(X_train.values)
    rmse = np.sqrt(-cross_val_score(model,X_train,Y_vals,scoring ="neg_mean_squared_error",cv=kf))
    return (rmse)
def rmse_CV_test(model):
    kf = KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model,X_train,Y_vals,scoring ="neg_mean_squared_error",cv=kf))
    return (rmse)

In [25]:
print('rmse on train',1-rmse_CV_train(etr_model).mean())
#print('rmse on train',rmse_CV_test(lr).mean())

rmse on train 0.828384748791


In [93]:
df_etr=pd.read_csv('./elastic_model.csv', index_col='Id')
df_rfr=pd.read_csv('./rfr_model.csv', index_col='Id')
df_lasso=pd.read_csv('./lasso_model.csv', index_col='Id')
df_elastic=pd.read_csv('./elastic_model.csv', index_col='Id')


In [94]:
df_average = (df_rfr+df_lasso+df_elastic+df_etr)/4
df_average.head()

,SalePrice
Id,
1461,123120.00125
1462,150738.73850
1463,177662.16450
1464,195197.55525
1465,194721.42850


In [95]:
df_average.to_csv('lasso_rfr_etr_elastic_average.csv')